In [1]:
%matplotlib inline
# since 2019.07.10
import os

from glob import glob
from multiprocessing import Pool

import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

In [2]:
n_jobs = 20

Read all scaled expression value

In [3]:
all_cell = glob(os.path.join("/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each", "*/scanpy"))
all_luad = glob(os.path.join("/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/LUAD", "*/scanpy"))
all_lusc = glob(os.path.join("/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/LUSC", "*/scanpy"))

all_cell

['/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/NK/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Dendritic/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Basal/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Neuroendocrine/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Alveolar_II/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Fibroblasts/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Erythroid_precursor/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Ciliated/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Exhaust_T/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/CD4/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/Monocytes/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots/03_each_cells/each/B_cells/scanpy',
 '/mnt/raid62/Lung_cancer_10x/final_plots

In [4]:
def read_data(path: str):
    u"""
    read scaled data
    """
    data = pd.read_csv(os.path.join(path, "scale.csv.gz"), index_col=0)
    meta = pd.read_csv(os.path.join(path, "meta.csv.gz"), index_col=0)
    
    # melt expression data
    data["gene"] = data.index
    data = data.melt(id_vars = ["gene"])
    
    # process meta info
    meta = meta.loc[:, ["res.0.6"]]
    meta["cell"] = meta.index
    
    # format data
    res = meta.merge(data, left_on="cell", right_on="variable")
    res = res.groupby(["res.0.6", "gene"]).mean()
    res = res.reset_index()
    res.columns = ["Cluster", "gene", "value"]
    res["Cell"] = os.path.basename(os.path.dirname(path))
    
    return res

Read data

In [5]:
with Pool(n_jobs) as p:
    all_cell = p.map(read_data, all_cell[:2])

MemoryError: 

In [50]:
pd.concat(all_cell).to_csv("/mnt/raid62/Lung_cancer_10x/final_plots/05_gene_drug/all_cell_scale_mean_expr.csv")

Read and process all luad data

In [ ]:
with Pool(n_jobs) as p:
    all_luad = p.map(read_data, all_luad)

In [53]:
pd.concat(all_luad).to_csv("/mnt/raid62/Lung_cancer_10x/final_plots/05_gene_drug/all_luad_scale_mean_expr.csv")

In [54]:
with Pool(n_jobs) as p:
    all_lusc = p.map(read_data, all_lusc)

In [55]:
pd.concat(all_lusc).to_csv("/mnt/raid62/Lung_cancer_10x/final_plots/05_gene_drug/all_lusc_scale_mean_expr.csv")